<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Circuit Stitching

H-System Quantum Credits (HQCs) are used to cost jobs. There are two components:

1. a +5 contribution for each job submitted to H-Series;
2. a job-specific cost accounting for the number of gate operations and shots.

\begin{equation}
HQC = 5 + \frac{C}{5000} \left( N_{1q} + 10 N_{2q} + 5 N_m \right)
\end{equation}

Circuit stitching is a method to "stitch" multiple jobs into 1 job using the Mid-Circuit Measurement and Reset (MCMR) feature. Stitching involves appending circuits depth-wise and width-wise (across the qubit register). To ensure execution and optimal performance, stitched circuits should be limited to tket programs of size ~1 MB, less then 10,000 2-qubit gate operations and a total cost of less then 30 HQCs.

**Contents:**
* [GHZ State Preparation](#GHZ-State-Preparation)
* [Parallelization Across Qubit Register](#Parallelisation-Across-Qubit-Register)
* [Circuit Stitching](#Circuit-Stitching)
* [Conclusion](#Conclusion)

## GHZ State Preparation

The $N$-qubit GHZ state preparation, using the definition,

\begin{equation}
| {GHZ}_N \rangle = \frac{1}{\sqrt{2}} \left( {| 0 \rangle}^{\bigotimes N} + {| 1 \rangle}^{\bigotimes N} \right),
\end{equation}

where $N=10$.

In [4]:
from pytket.circuit import Circuit

n_qubits = 10
circ = Circuit(n_qubits)
circ.H(0)
for i in range(n_qubits-1):
    circ.CX(i,i+1);

## Parallelization Across Qubit Register

10 replicas of the GHZ state-preparation circuit are stored inside a `pytket.circuit.CircBox` instance.

In [5]:
from pytket.circuit import CircBox

ghz_subcircuits = [CircBox(circ.copy()) for _ in range(10)]

In [6]:
from typing import List
import numpy as np
from pytket.circuit import (
    Circuit, 
    CircBox,
    Qubit
)

def parallelize_subcircuits(
    circbox_list: List[CircBox],
    device_qubits: int,
    n: int
) -> Circuit:
    r"""Parallelize subcircuit across device register.
    :param circbox_list: 
    :param_type: List[CircBox]
    :param device_qubits: 
    :param_type: int
    :param n: size of subcircuit register
    :param_type: int
    """
    p = np.floor_divide(device_qubits, n)
    qubits = [[Qubit(i) for i in range(q, q+n)] for q in range(0, device_qubits, n)]
    circuit = Circuit(device_qubits)
    for i in range(0, device_qubits+p, p):
        for j, (c, q_list) in enumerate(zip(circbox_list[i:i+p], qubits)):
            circuit.add_circbox(c, q_list)
            creg = circuit.add_c_register(f"creg_{i}_{j}", len(q_list))
            for q, cb in zip(q_list, creg):
                circuit.Measure(q, cb)
                circuit.Reset(q)
    return circuit

A 10-qubit GHZ state-preparation replicas can be parallelized twice across H1-1E (20-qubit device). The function below fits the 10 replicas on the H1-1E register.

In [9]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

In [10]:
device_nqubits = backend.backend_info.n_nodes

In [11]:
circuit1 = parallelize_subcircuits(ghz_subcircuits, device_nqubits, n_qubits)

In [15]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit1)

## Circuit Stitching

The circuit above parallelizes 10 GHZ circuits, each of width 10-qubits, across a 20-qubit register. This is now stitched 40 times. This stitched circuit is equivalent to measuring the GHZ state in the computational basis with 400 shots.

In [20]:
from pytket.circuit import Circuit

def stitch_circuit(
    circuit: Circuit, 
    s: int
) -> Circuit:
    stitched_circuit = Circuit(circuit.n_qubits)
    box = CircBox(circuit)
    for i in range(s):
        creg = stitched_circuit.add_c_register(f"creg_{i}", circuit.n_bits)
        stitched_circuit.add_circbox(box, stitched_circuit.qubits + list(creg))
    return stitched_circuit

In [21]:
circuit2 = stitch_circuit(circuit1, 40)

## Job Submission

In [23]:
compiled_circuit = backend.get_compiled_circuit(circuit2, optimisation_level=2)

In [25]:
compiled_circuit.n_2qb_gates()

3600

In [31]:
from pytket.extensions.quantinuum import Language

handle = backend.process_circuit(compiled_circuit, n_shots=1, syntax_checker="H1-1SC", language=Language.QIR)

ValueError: classical registers must not have more than 64 bits, you could try to set cut_pytket_register=True in the conversion

## Conclusion

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>